## Voronoi constructor Module 

### Usefull Libraries

In [1]:
import math
import numpy as np
import sys

import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.colors as mcolors
import matplotlib._color_data as mcd

from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay

import import_ipynb
import node_linker
import solver

importing Jupyter notebook from node_linker.ipynb
importing Jupyter notebook from solver.ipynb
That is:  [-1.39306177  2.35818188]


### Position Generation

`position_generator(num_points, seed)`

Generate a pseudorandom configurations for initial and final positions

#### Parameters:
* **num_points&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Number of robots
* **seed&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Seed for pseudorandom
    
#### Return:
* **robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;array of list of floats**
    * Initial coordinates for robots position
* **target&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;array of list of floats**
    * Final coordinates for robots position
* **distances&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of floats**
    * Distances between initial and final positions

In [16]:
def position_generator(num_points, seed):
    robots = []
    target = []
    distances = []
    radius = math.sqrt(2*num_points)
    np.random.seed(seed)
    offset = (1/(0.4*num_points) * np.random.random_sample((num_points*4, ))
              - 1/(0.4*num_points))
    for i in range(0,4*num_points,4):
        convertion = (2*math.pi*i/4)/num_points
        robots.append([round(radius*math.cos(convertion)+offset[i],2),
                       round(radius*math.sin(convertion)+offset[i+1],2)])
        target.append([round(radius*math.cos(convertion+math.pi)+offset[i+2],
                             2),
                       round(radius*math.sin(convertion+math.pi)+offset[i+3],
                             2)])
    robots = np.array(robots)*10
    target = np.array(target)*10
    for i in range(num_points):
        distances.append(np.linalg.norm(target[i] - robots[i]))
    return robots, target, distances

### Sharpen Function

`def sharpen(head, tail)`

Calculate a new vertex when edges smaller than the safety radius appear 

#### Parameters:
* **aux1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Auxiliary coordinate for vertex1 to generate the peak 
* **vertex1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * One of the edge vertices
* **vertex2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * The other edge vertices
* **aux2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Auxiliary coordinate for vertex2 to generate the peak
    
#### Return:
* **peak&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * New coordinate that replace the edge

In [ ]:
def sharpen(aux1, vertex1, vertex2, aux2):
    # Special cases: vertical edge
    if aux1[0] == vertex1[0]:
        slope2 = (vertex2[1] - aux2[1])/(vertex2[0] - aux2[0])
        x = aux1[0]
        y = slope2*(x - aux2[0]) + aux2[1]
    elif aux2[0] == vertex2[0]:
        slope1 = (vertex1[1] - aux1[1])/(vertex1[0] - aux1[0])
        x = aux2[0]
        y = slope1*(x - aux1[0]) + aux1[1]
    # Another cases
    else:
        slope1 = (vertex1[1] - aux1[1])/(vertex1[0] - aux1[0])
        slope2 = (vertex2[1] - aux2[1])/(vertex2[0] - aux2[0])
        x = ((vertex2[1] - aux1[1] + slope1*aux1[0] - slope2*vertex2[0]) /
             (slope1 - slope2))
        y = slope1*(x - aux1[0]) + aux1[1]
    peak = np.array([x,y])
    return peak

### Collision Free Configuration Function

`def collision_free_conf(positions, safety_radius)`

Test if the initial position of robots is collision free configuration.

#### Parameters:
* **positions&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * Positions of robots 
* **pointers&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of ints, shape `(nridges, 2)`**
    * Indices of the points between which each Voronoi ridge lies (from scipy.spatial.Voronoi.ridge_points)
* **safety_radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
    
#### Return:
* **no_collision&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * True if no collision exists

In [73]:
def collision_free_conf(positions, pointers, safety_radius):
    no_collision = True
    i = 0
    while i < len(pointers) and no_collision:
        if np.linalg.norm(positions[pointers[i][1]]
                          - positions[pointers[i][0]]) < 2*safety_radius:
            no_collision = False
        i += 1
    return no_collision

### Vertices Sense Function

`def getSense(vertices, previous_position)`

Get the sense of a vertices set

#### Parameters:
* **vetices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarrays of float**
    * Current positions of the robots.
* **previous_position&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Coordinates of the previous positions of robots
    
#### Return:
* **sense&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Order sense of vertices 

In [ ]:
def getSense(vertices, previous_position):
    # Check the type of cell
    if len(vertices) > 2:
        midpoint = vertices[1]
        extreme1 = vertices[0] - midpoint
        extreme2 = vertices[2] - midpoint
    else:
        midpoint = previous_position
        extreme1 = vertices[0] - midpoint
        extreme2 = vertices[1] - midpoint
    # Analyze the crossproduct
    crossproduct = extreme1[0]*extreme2[1] - extreme2[0]*extreme1[1]
    if crossproduct > 0:
        sense = 'clockwise'
    elif crossproduct < 0:
        sense = 'counterclockwise'
    return sense

### Point in a Segment Function

`def isBetween(a, b, c)`

Check inclusion of a point in a segment

#### Parameters:
* **a&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of float**
    * First vertex of the segment.
* **b&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of float**
    * Second vertex of the segment.
* **c&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of float**
    * Point to check.    
#### Return:
*  **True/False depending on the inclusion of the point inside the line**

In [ ]:
def isBetween(a, b, c):
    crossproduct = (c[1] - a[1])*(b[0] - a[0]) - (c[0] - a[0])*(b[1] - a[1])
    # compare versus epsilon for floating point values, or != 0 if using
    # integers
    if abs(crossproduct) > 0.01:
        return False
    dotproduct = (c[0] - a[0])*(b[0] - a[0]) + (c[1] - a[1])*(b[1] - a[1])
    if dotproduct < 0:
        return False
    squaredlengthba = (b[0] - a[0])*(b[0] - a[0]) + (b[1] - a[1])*(b[1] - a[1])
    if dotproduct > squaredlengthba:
        return False
    return True

### Deadlock Function

`def deadlock(robots, new_positions, vertices_bvc, edges, delta, previous_positions)`

Check if there are robots in deadlock situation

#### Parameters:
* **robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Current positions of the robots.
* **new_positions&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Positions to move the robots.
* **vertices_bvc&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of lists of ndarrays of floats**
    * Coordinates (floats) of BVC vertices_bvc (ndarray) for each cell (list)
* **delta&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * Amount of space that robot will move if deadlock occurs
* **previous_position&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Coordinates of the previous positions of robots
* **deadlock_pointers&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of ints**
    * Pointers to robots in deadlock
* **deadlock_tolerance&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * Maximum distance to determine deadlock
* **closer_edges&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of lists of ndarrays of ints**
    * Edges closer to deadlock
* **cycles&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Number of previous positions to take into account to break deadlock
* **alpha&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * Simple movement of the robot
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Flag for verbose
    
#### Return:
* **fixed_up_positions&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Positions to avoid deadlock

In [ ]:
def deadlock(robots, new_positions, vertices_bvc, delta, previous_positions,
             deadlock_pointers, deadlock_tolerance, closer_edges, cycles,
             alpha, verbose):
    # List to return
    fixed_up_positions = []
    # Check all robots positions
    for pointer in range(len(robots)):
        # Check if the robot is in deadlock situation
        if pointer in deadlock_pointers:
            # Initialize the instantaneous set of BVC vertices
            inst_vertices = vertices_bvc[pointer]
            # Get the sense of the vertices_bvc array
            sense = getSense(inst_vertices, previous_positions[-2][pointer])
            #i = 0
            milestone = -1
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Robot in deadlock:',robots[pointer])
                print('Vertices to check:',inst_vertices)
                print('Sense:',sense)
            main_distance = np.linalg.norm(new_positions[pointer]
                                           - inst_vertices[0])
            for i in range(len(inst_vertices)):
                temp_distance = np.linalg.norm(new_positions[pointer]
                                               - inst_vertices[i])
                if verbose:
                    print('Distance from vertex to the closest point:',
                          temp_distance)
                if (temp_distance < deadlock_tolerance
                    and temp_distance < main_distance):
                    milestone = i
                    main_distance = temp_distance
                j = 0
                for j in range(cycles):
                    previous_distance = np.linalg.norm(inst_vertices[i]
                                               - previous_positions[-2-j][pointer])
                    if verbose:
                        print('Distance from the '+str(j)
                              +'th previus point:',previous_distance)
                    if (previous_distance < deadlock_tolerance
                        and previous_distance < main_distance):
                        milestone = i
                        main_distance = previous_distance
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Pointer:',milestone)
            if milestone > -1 and milestone < len(inst_vertices):
                #------------------------CONTROL POINT------------------------
                if verbose:
                    print('Deadlock in vertex')
                if sense == 'counterclockwise':
                    if milestone == 0:
                        aux_vertex = (inst_vertices[-2]
                                      - inst_vertices[milestone])
                    else:
                        aux_vertex = (inst_vertices[milestone-1]
                                      - inst_vertices[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = aux_vertex*delta + robots[pointer]
                elif sense == 'clockwise':
                    aux_vertex = (inst_vertices[milestone+1]
                                  - inst_vertices[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = aux_vertex*delta + robots[pointer]
            # If deadlock is on an edge
            else:
                #------------------------CONTROL POINT------------------------
                if verbose:
                    print('Deadlock in edge')
                if sense == 'clockwise':
                    aux_vertex = closer_edges[2*pointer] - robots[pointer]
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = aux_vertex*delta + robots[pointer]
                elif sense == 'counterclockwise':
                    aux_vertex = closer_edges[2*pointer+1] - robots[pointer]
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = aux_vertex*delta + robots[pointer]
            # Make sure the point is inside BVC
            if path.Path(inst_vertices).contains_point(position_candidate):
                fixed_up_positions.append(position_candidate)
            else:
                fixed_up_positions.append(robots[pointer])
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('-----------------------------------')
        else:
            fixed_up_positions.append(new_positions[pointer])
    #------------------------------CONTROL POINT------------------------------
    return np.array(fixed_up_positions)

### Intersection Function

`def intersect(vertice1, origin, vertice2, safety_radius):`

Generate the new BVC vertices from the intersection of the Voronoi edges  
moved according the safety radius in different sectors of the cell

#### Parameters:
* **vertice1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme Voronoi vertex.
* **origin&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector intermediate Voronoi vertex.
* **vertice2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme Voronoi vertex.
* **safety_radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram

#### Return:
* **new_vertice1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme BVC vertex.
* **intersection&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector intermediate BVC vertex.
* **new_vertice2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme BVC vertex.

In [4]:
def intersect(vertice1, origin, vertice2, safety_radius):
    # Change coordinates
    vertice1 = vertice1 - origin
    vertice2 = vertice2 - origin
    # Unit vectors
    vertice1_unit = vertice1 / np.linalg.norm(vertice1)
    vertice2_unit = vertice2 / np.linalg.norm(vertice2)
    # Get the angle between edges
    p_dot = np.dot(vertice1_unit, vertice2_unit)
    angle = np.arccos(p_dot)
    mid_angle = angle/2
    # Compute and show the intersection of the BVC
    mid_vertex = (vertice1_unit + vertice2_unit) / 2
    mid_vertex_unit = mid_vertex / np.linalg.norm(mid_vertex)
    separation = safety_radius / np.sin(mid_angle)
    intersection = mid_vertex_unit * separation
    # Revert coordinates
    new_vertice1 = vertice1 + intersection + origin
    new_vertice2 = vertice2 + intersection + origin
    intersection = intersection + origin
    return [new_vertice1, intersection, new_vertice2]

### Closest Point Function

`def search_closest_point(target_point, vertices)`

Search the closest point for a robot in its BV cell. This closest point  
is either a vertex or a edge if the direction from the robot to the  
target position is perpendicular to this edge.

#### Parameters:
* **target_point&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * The target point of a specific robot (It should be a global variable).
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Finite BVC vertices.
    
#### Return:
* **vertices\[i\]&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Closest vertex of the BVC to the target

In [64]:
def search_closest_point(target_point, vertices):
    # Initialize
    closest_point = np.linalg.norm(vertices[0] - target_point)
    pointer = 0
    # Review the rest of vertices
    for i in range(1,len(vertices)):
        # Measure the distance of the candidate vertex to the target
        candidate = np.linalg.norm(vertices[i] - target_point)
        # Take the closest point
        if candidate < closest_point:
            closest_point = candidate
            pointer = i
    # Return the closest vertex 
    return vertices[pointer]

### Closer Edge Points Function

`def perpendicular_distance(target_point, vertices)`

Get the closest point of an edge to the target point

#### Parameters:
* **target_point&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * The target point of a specific robot.
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Finite BVC vertices.
    
#### Return:
* **edge_points&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * List of closest points on the BVC edges to the target

In [ ]:
def perpendicular_distance(target_point, vertices):
    #Initialize the list to return
    edge_points = []
    # Analize each edge
    for i in range(len(vertices[:-1])):
        # Check if the edge is vertical
        if vertices[i][0] == vertices[i+1][0]:
            [x, y] = [vertices[i][0], target_point[1]]
        else:
            # Get the perpendicular point on the egde sense to the target
            slope = (vertices[i+1][1] - vertices[i][1])/(vertices[i+1][0]
                                                         - vertices[i][0])
            x = (slope * (vertices[i][1] - target_point[1]
                          - slope * vertices[i][0])
                 - target_point[0]) / (-1 - pow(slope,2))
            y = (x - target_point[0])/(-slope) + target_point[1]
            # Check if the point is inside of edge to include in the returning
            # list
        if isBetween(vertices[i], vertices[i+1], [x,y]):
            edge_points.append([x,y])
    return edge_points

### Critical Distance Function

`def approved_distance(node, safety_radius, vertices, sense)`

Check if the safety radius length does not create conflicts with the edge lenght

#### Parameters:
* **node&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;node_linker.Node**
    * The start node of the analysis that points to a Voronoi vertex in a determined cell.
* **safety_radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Updated vertices of Voronoi Diagram with approximation of infinity vertices
* **sense&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;string**
    * Back or forward sense

#### Return:
* **approve&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;boolean**
    * True if there is not a problem

In [ ]:
def approved_distance(node, safety_radius, vertices, sense, verbose):
    coordinate2 = vertices[node.item]
    # Primarly for open cells
    if sense == 'forward':
        coordinate1 = vertices[node.pref.item]
        coordinate3 = vertices[node.nref.item]
        coordinate4 = vertices[node.nref.nref.item]
    # For closed cells
    elif sense == 'back':
        coordinate1 = vertices[node.nref.item]
        coordinate3 = vertices[node.pref.item]
        coordinate4 = vertices[node.pref.pref.item]  
    # Use the Intersections Formula and Sharpen Function in order to reach the
    # critical distance
    inst1 = intersect(coordinate1, coordinate2, coordinate3, safety_radius)
    inst2 = intersect(coordinate2, coordinate3, coordinate4, safety_radius)
    critical_point = sharpen(coordinate2, inst1[1], coordinate3, inst2[1])
    approve = not isBetween(coordinate2, inst1[1], critical_point)
    if verbose:
        print('\nChecking distances in',sense,'sense')
        print('Critical point:',critical_point)
        print('First segment: {',coordinate2,',',inst1[1],'}')
        print('Second segment: {',coordinate3,',',inst2[1],'}')
        print('Approve?',approve,'\n')
    return approve

### Pruner Function

`def pruner(vertices)`

Remove intersections of the BVC

#### Parameters:
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * BVC vertices
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;boolean**
    * Flag to print important values

#### Return:
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * BVC vertices without intersections

In [ ]:
def pruner(vertices, verbose):
    pruned_vertices = []
    cell = node_linker.DoublyLinkedList()
    for vertex in vertices:
        cell.insert_at_end(vertex)
    runner = cell.start_node
    if verbose:
        print('Vertices in doubly linked list:')
        cell.traverse_list()
    while not(runner is cell.end_node.pref.pref
          or runner is cell.end_node.pref):
        vanguard = runner.nref.nref
        while vanguard is not cell.end_node:
            vertex_0 = runner.item
            vertex_1 = runner.nref.item
            vertex_2 = vanguard.item
            vertex_3 = vanguard.nref.item
            if verbose:
                print("\nChecking: ",vertex_0,"—",vertex_1," and ",vertex_2
                      ,"—",vertex_3)
            critical_point = sharpen(vertex_0, vertex_1, vertex_2, vertex_3)
            intersection = isBetween(vertex_0, vertex_1, critical_point)
            if verbose:
                print('Intersection?:',intersection)
            if intersection:
                runner.nref.item = critical_point
                runner.nref.nref = vanguard.nref
                vanguard.nref.pref = runner.nref
                if verbose:
                    print('The new value is: ',runner.nref.item)
                    print('Connected with: ',runner.nref.nref.item)
            vanguard = vanguard.nref
        runner = runner.nref
    runner = cell.start_node
    while runner is not cell.end_node.nref:
        pruned_vertices.append(runner.item)
        runner = runner.nref
    if verbose:
        print("Number of pruned vertices: ",str(cell.lenght_list))
    return pruned_vertices

### Voronoi Diagram Generator Method

`def voronoi_approx(robots, vor, verbose)`

Generate the main Voronoi Diagram approximating the values of infinity vertices  
and returning the updated values of edges and vertices for BVC constructor.

#### Parameters:
* **robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Robots position
* **vor&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;class scipy.spatial.Voronoi**
    * Voronoi diagram in N dimension
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Flag for verbose
    
#### Return:
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Updated vertices with approximation of infinity vertices
* **edges&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of int**
    * Updated pointers to vertices for each edge

In [40]:
def voronoi_approx(robots, vor, verbose):
    counter = 0
    # Create copies of:
    vertices = vor.vertices
    edges = []
    # Find the center of the robots position
    center = robots.mean(axis=0)
    # pointidx: Indices of the points between which each Voronoi ridge lies.
    # simplex: Indices of the Voronoi vertices forming each Voronoi ridge.
    # zip join each values in a tuple
    for pointidx, simplex in zip(vor.ridge_points, vor.ridge_vertices):
        # Transform to array
        simplex_arr = np.array(simplex)
        # If the Voronoi edge goes to infinity
        if np.all(simplex_arr >= 0):
            # Append to the BVC edges list
            edges.append(simplex)
        # If one of the vertices goes to the infinity
        else:
            # Get the finite vertex
            i = simplex_arr[simplex_arr >= 0][0] 
            # Calculate the straight line between robots
            t = robots[pointidx[1]] - robots[pointidx[0]] 
            # Normalization of this line
            t = t / np.linalg.norm(t)
            # Ortogonalization
            n = np.array([-t[1], t[0]])
            # Get the midpoint between robots
            midpoint = robots[pointidx].mean(axis=0)
            # Far point = extension from the finite vertex using the
            #             orthogonalization and
            #             depending of the midpoint location respect center
            far_point = vor.vertices[i] + (np.sign(np.dot(midpoint- center,n))
                                           * n * 20 * len(robots))
            # Increase the counter
            counter += 1
            # Add the value to the BVC vertices array
            vertices = np.concatenate((vertices, far_point.reshape(1,2)),
                                      axis=0)
            # Update the pointer of BVC edges list 
            edges.append([len(vertices)-1, simplex[1]])
    if verbose:
        print('List of vertex coordinates (including approximations) of',
              'Voronoi Diagram:\n',vertices,'\n')
        print('Robots in front: ',vor.ridge_points,'\n')
        print('Voronoi Diagram Edges: ',edges,'\n')
    #print('+++++++++++++++++++++++++++++++++++++++')
    return [vertices, edges]

### Vertices Assembler Function

`def vertices_assembler(robots, vor, edges, verbose)`

Assemble the vertices for each cell in order to generate the new BVC vertices

#### Parameters:
* **robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Robots position
* **vor&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;class scipy.spatial.Voronoi**
    * Voronoi diagram in N dimension
* **edges&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of int**
    * Updated pointers to vertices for each edge
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Flag for verbose
    
#### Return:
* **bvc&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of node_linker.DoublyLinkedList**
    * A list of ordered vertices for each robot

In [41]:
def vertices_assembler(robots, vor, edges, verbose):
    bvc = []
    # Initialize cells (double linked lists)
    for i in range(len(robots)):
        cell = node_linker.DoublyLinkedList()
        bvc.append(cell)
    if verbose:
        print('::::::::::::::::::::::::::::::')
        print('START ASSEMBLY!\n')
        print('Number of cells:',len(bvc),'\n')
    #      vertex
    # robot  |  robot
    #     vertex
    ridge_list = vor.ridge_points.tolist()
    for front, edge in zip(ridge_list, edges):
        if verbose:
            print('******************************')
            print('Edge:',edge,'; Robots in front:',front,'\n')
        # For each robot
        for i in range(len(front)):
            # Get the values
            double_linked_list = bvc[front[i]]
            # CHECK VERTEX SITUATION 
            # flag = ['case', node, position]
            flag0 = double_linked_list.search_list(edge[0])
            flag1 = double_linked_list.search_list(edge[1])
            vertex0 = {"value" : edge[0],
                       "case"  : flag0[0],
                       "node"  : flag0[1],
                       "position" : flag0[2] 
                      }
            vertex1 = {"value" : edge[1],
                       "case"  : flag1[0],
                       "node"  : flag1[1],
                       "position" : flag1[2] 
                      }
            if verbose:
                print('Robot N°',front[i],'\n')
                print('Analysis of vertex',vertex0["value"],'in the cell:',
                      vertex0["case"])
                print('Analysis of vertex',vertex1["value"],'in the cell:',
                      vertex1["case"],'\n')
            # CASES
            # CASE 1: CREATE LIST
            if vertex0["case"] == vertex1["case"] == 'empty':
                double_linked_list.insert_in_emptylist(vertex0["value"])
                double_linked_list.insert_at_end(vertex1["value"])
                double_linked_list.start_node.nlink = True
                double_linked_list.end_node.plink = True
                message = ['Double linked list created!', vertex0["value"],
                           vertex1["value"]]
            # CASE 2: PARTIAL INSERTION
            elif vertex0["case"] == vertex1["case"] == 'none':
                double_linked_list.insert_at_end(vertex0["value"])
                double_linked_list.insert_at_end(vertex1["value"])
                double_linked_list.end_node.pref.nlink = True
                double_linked_list.end_node.plink = True
                message = ['Partial insertion!', vertex0["value"], vertex1["value"]]
            # THE FIRST VERTEX IS NOT IN THE LIST AND...
            elif vertex0["case"] == 'none':
                # CASE 3: THE SECOND VERTEX IS AT THE START OF THE LIST
                if vertex1["case"] == 'space before':
                    double_linked_list.insert_at_start(vertex0["value"])
                    double_linked_list.start_node.nlink = True
                    vertex1["node"].plink = True
                    message = ['Insertion at start!', vertex0["value"],
                               vertex1["value"]]
                # CASE 4: THE SECOND VERTEX IS INSIDE OF THE LIST
                elif vertex1["case"] == 'inside':
                    if not vertex1["node"].plink:
                        double_linked_list.insert_before_item(vertex1["value"],
                                                              vertex0["value"])
                        vertex1["node"].pref.nlink = True
                        vertex1["node"].plink = True
                        message = ['Insertion inside before item!',
                                   vertex0["value"], vertex1["value"]]
                    else:
                        double_linked_list.insert_after_item(vertex1["value"],
                                                             vertex0["value"])
                        vertex1["node"].nref.plink = True
                        vertex1["node"].nlink = True
                        message = ['Insertion inside after item!',
                                   vertex1["value"], vertex0["value"]]
                # CASE 5: THE SECOND VERTEX IS AT THE END OF THE LIST
                elif vertex1["case"] == 'space after':
                    double_linked_list.insert_at_end(vertex0["value"])
                    double_linked_list.end_node.plink = True
                    vertex1["node"].nlink = True
                    message = ['Insertion at end!', vertex1["value"],
                               vertex0["value"]]
            # THE FIRST VERTEX IS AT THE START OF THE LIST AND...
            elif vertex0["case"] == 'space before':
                # CASE 6: THE SECOND VERTEX IS NOT IN THE LIST
                if vertex1["case"] == 'none':
                    double_linked_list.insert_at_start(vertex1["value"])
                    vertex0["node"].plink = True
                    double_linked_list.start_node.nlink = True
                    message = ['Insertion at start!', vertex1["value"],
                               vertex0["value"]]
                # CASE 7: THE SECOND VERTEX IS INSIDE OF THE LIST
                elif vertex1["case"] == 'inside':
                    double_linked_list.reverse_linked_list(vertex0["node"],
                                                           vertex1["node"].pref)
                    vertex0["node"].nlink = True
                    vertex1["node"].plink = True
                    message = ['Reorganization!', vertex0["value"],
                               vertex1["value"]]
                # CASE 8: THE SECOND VERTEX IS AT THE END OF THE LIST
                elif vertex1["case"] == 'space after':
                    runner = vertex0["node"].nref
                    connected = runner.plink and runner.nlink
                    while (connected
                           and runner.nref is not double_linked_list.end_node):
                        runner = runner.nref
                        connected = runner.plink and runner.nlink
                    # CASE 8a: TIME TO CLOSE RING
                    if connected:
                        vertex0["node"].pref = vertex1["node"]
                        vertex1["node"].nref = vertex0["node"]
                        vertex0["node"].plink = True
                        vertex1["node"].nlink = True
                        message = ['Ring closed!', vertex1["value"],
                                   vertex0["value"]]
                    # CASE 8b: DOUBLE REORGANIZATION
                    else:
                        unchain_node1 = runner
                        unchain_node2 = runner.nref
                        double_linked_list.reverse_linked_list(vertex0["node"],
                                                               unchain_node1)
                        double_linked_list.reverse_linked_list(unchain_node2,
                                                               vertex1["node"])
                        vertex0["node"].nlink = True
                        vertex1["node"].plink = True
                        message = ['Double reorganization!', vertex0["value"],
                                   vertex1["value"]]
            # THE FIRST VERTEX IS INSIDE OF THE LIST AND...
            elif vertex0["case"] == 'inside':
                # CASE 9: THE SECOND VERTEX IS NOT IN THE LIST
                if vertex1["case"] == 'none':
                    if not vertex0["node"].plink:
                        double_linked_list.insert_before_item(vertex0["value"],
                                                              vertex1["value"])
                        vertex0["node"].plink = True
                        vertex0["node"].pref.nlink = True
                        message = ['Insertion inside! before item!',
                                   vertex1["value"], vertex0["value"]]
                    else:
                        double_linked_list.insert_after_item(vertex0["value"],
                                                             vertex1["value"])
                        vertex0["node"].nlink = True
                        vertex0["node"].nref.plink = True
                        message = ['Insertion inside after item!',
                                   vertex0["value"], vertex1["value"]]                
                # CASE 10: THE SECOND VERTEX IS AT THE START OF THE LIST
                elif vertex1["case"] == 'space before':
                    double_linked_list.reverse_linked_list(vertex1["node"],
                                                           vertex0["node"].pref)
                    vertex0["node"].plink = True
                    vertex1["node"].nlink = True
                    message = ['Reorganization!', vertex1["value"],
                               vertex0["value"]]
                # CASE 11: THE SECOND VERTEX IS AT THE END OF THE LIST
                elif vertex1["case"] == 'space after':
                    # CASE 11a: WEAK CHAIN
                    if not vertex0["node"].plink:
                        runner = vertex0["node"].nref
                        while runner.nlink:
                            runner = runner.nref
                        aux = runner.nref
                        double_linked_list.reverse_linked_list(vertex0["node"],
                                                               runner)
                        double_linked_list.reverse_linked_list(aux,
                                                               vertex1["node"])
                        vertex0["node"].nlink = True
                        vertex1["node"].plink = True
                        message = ['Double reorganization!', vertex0["value"],
                               vertex1["value"]]
                    # CASE 11b: ONE REORGANIZATION
                    else:
                        double_linked_list.reverse_linked_list(vertex0["node"].nref,
                                                               vertex1["node"])
                        vertex0["node"].nlink = True
                        vertex1["node"].plink = True
                        message = ['Reorganization!', vertex0["value"],
                                   vertex1["value"]]
                # CASE 12: THE SECOND VERTEX IS INSIDE OF THE LIST
                elif vertex1["case"] == 'inside':
                    #CASE 12a: THE POSITION IS CORRECT
                    if vertex0["node"].nref == vertex1["node"]:
                        vertex0["node"].nlink = True
                        vertex1["node"].plink = True
                        message = ['Already done!', vertex0["value"],
                                   vertex1["value"]]
                    elif vertex0["node"].pref == vertex1["node"]:
                        vertex0["node"].plink = True
                        vertex1["node"].nlink = True
                        message = ['Already done!', vertex0["value"],
                                   vertex1["value"]]
                    else:
                        #CASE 12b: THE POSITION IS INCORRECT
                        ridge_list.append(front)
                        edges.append(edge)
                        message = ['Postponed!', vertex0["value"], vertex1["value"]]
            # THE FIRST VERTEX IS AT THE END OF THE LIST AND...
            elif vertex0["case"] == 'space after':
                # CASE 13: THE SECOND VERTEX IS NOT IN THE LIST
                if vertex1["case"] == 'none':
                    double_linked_list.insert_at_end(vertex1["value"])
                    vertex0["node"].nlink = True
                    double_linked_list.end_node.plink = True
                    message = ['Insertion at end!', vertex0["value"],
                               vertex1["value"]]
                # CASE 14: THE SECOND VERTEX IS AT THE START OF THE LIST
                elif vertex1["case"] == 'space before':
                    runner = vertex1["node"].nref
                    connected = runner.plink and runner.nlink
                    while (connected
                           and runner.nref is not double_linked_list.end_node):
                        runner = runner.nref
                        connected = runner.plink and runner.nlink
                    # CASE 14a: TIME TO CLOSE RING
                    if connected:
                        vertex1["node"].pref = vertex0["node"]
                        vertex0["node"].nref = vertex1["node"]
                        vertex1["node"].plink = True
                        vertex0["node"].nlink = True
                        message = ['Ring closed!', vertex0["value"],
                                   vertex1["value"]]
                    # CASE 14b: DOUBLE REORGANIZATION
                    else:
                        unchain_node1 = runner
                        unchain_node2 = runner.nref
                        double_linked_list.reverse_linked_list(vertex1["node"],
                                                               unchain_node1)
                        double_linked_list.reverse_linked_list(unchain_node2,
                                                               vertex0["node"])
                        vertex1["node"].nlink = True
                        vertex0["node"].plink = True
                        message = ['Double reorganization!', vertex1["value"],
                                   vertex0["value"]]
                # CASE 15: THE SECOND VERTEX IS INSIDE OF THE LIST
                elif vertex1["case"] == 'inside':
                    # CASE 15a: WEAK CHAIN
                    if not vertex1["node"].plink:
                        runner = vertex1["node"].nref
                        while runner.nlink:
                            runner = runner.nref
                        aux = runner.nref
                        double_linked_list.reverse_linked_list(vertex1["node"],
                                                               runner)
                        double_linked_list.reverse_linked_list(aux,
                                                               vertex0["node"])
                        vertex1["node"].nlink = True
                        vertex0["node"].plink = True
                        message = ['Double reorganization!', vertex1["value"],
                               vertex0["value"]]
                    # CASE 15b: ONE REORGANIZATION
                    else:
                        double_linked_list.reverse_linked_list(vertex1["node"].nref,
                                                               vertex0["node"])
                        vertex0["node"].plink = True
                        vertex1["node"].nlink = True
                        message = ['Reorganization!', vertex1["value"],
                                   vertex0["value"]]
            if verbose:
                print(message[0],message[1],']---[',message[2])
                double_linked_list.traverse_list()
                print('Head:',double_linked_list.start_node.item)
                print('Tail:',double_linked_list.end_node.item)
                print('---------------------')
    if verbose:
        print('\n::::::::::::::::::::::::::::::\n')    
    return bvc

### Creating the palette

`def colors(number):`

Create the color palette

#### Parameters:
* **number&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Number of colors to retun
    
#### Return:
* **palette&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of str**
    * A list with the colors name

In [1]:
def colors(number):
    overlap = ['red','tomato','orangered','orange','goldenrod','gold',
               'yellowgreen','chartreuse','lime','green','darkgreen','teal',
               'turquoise','aqua','cyan','blue','navy','indigo','purple',
               'fuchsia','magenta','crimson','maroon','brown','sienna']
    palette = []
    for i in range(number):
        palette.append(overlap[int(i*(len(overlap)/number))])
    return palette


### BVC Generator Function

`def bvc_generator(robots, target, safety_radius, vertices, edges, bvc, verbose)`

Assemble the vertices for each cell in order to generate the new BVC vertices

#### Parameters:
* **robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Robots position
* **target&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Final position of robots
* **safety_radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Updated vertices with approximation of infinity vertices
* **edges&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of int**
    * Updated pointers to vertices for each edge
* **bvc&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of node_linker.DoublyLinkedList**
    * A list of ordered vertices for each robot
* **optimize&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Flag for optimization
* **plotting&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Flag for plotting
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Flag for verbose
    
#### Return:
* **new_robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * New position of robots

In [2]:
def bvc_generator(robots, target, safety_radius, vertices, edges, bvc,
                  robots_path, optimize, plotting, verbose):
    counter = 0
    palette = colors(len(robots))
    new_robots = [] # For storing the next position of robots
    set_of_new_vertices = [] # For storing the BVC vertices of each robot
    closer_edges = [] # For save the closer edge
    # For internal color
    if plotting:
        fig, ax = plt.subplots()
    # Calculate the bvc vertices and the closest point
    for cell in bvc:
        if verbose:
            print('Analyzing these vertices...')
            cell.traverse_list()
        new_vertices = []
        milestone_node = cell.start_node
        # Go in pairs of edges
        # Check if the cell is unlimited
        if cell.end_node.nref is None:
            if verbose:
                print('OPEN CELL!')
                print('Number of vertices in this cell:',
                      cell.lenght_list,'\n')
            while milestone_node.nref.nref is not cell.end_node.nref:
                # Auxiliar node for possible cuts
                aux_node = milestone_node.nref
                # Get coordinates
                vertex_start = vertices[milestone_node.item]
                vertex_mid1 = vertices[milestone_node.nref.item]
                vertex_mid2 = vertices[aux_node.item]
                vertex_end = vertices[aux_node.nref.item]
                # Get the new vertices in this sector
                sector = intersect(vertex_start, vertex_mid1, vertex_end,
                                   safety_radius)
                # Assign the new values according special cases 
                if milestone_node is cell.start_node:
                    new_vertices.append(sector[0])
                new_vertices.append(sector[1])
                if aux_node.nref.nref is None:
                    new_vertices.append(sector[2])
                # Move the milestone_node 
                milestone_node = aux_node
            # Remove intersections of the BVC
            new_vertices = pruner(new_vertices,verbose)
        # If the cell is limited or closed
        else:
            # Simple counter for jumps
            i = 0
            if verbose:
                print('CLOSED CELL!')
                print('Number of vertices in this cell:',cell.lenght_list,
                      '\n')
            while i < cell.lenght_list:
                # Flags for jumps back and forward
                flag_jumps_back = False
                flag_jumps_forward = False
                # Establish the auxiliar nodes to move back and forward
                aux_node_back = milestone_node
                aux_node_forward = milestone_node
                if verbose:
                    print('Checking this milestone (vertex):',
                          milestone_node.item)
                # Compare the safety radius with the critical distance for the
                # closest edge (back)
                while not approved_distance(aux_node_back, safety_radius,
                                            vertices, 'back', verbose):
                    # Update the node, jumps and flag
                    aux_node_back = aux_node_back.pref
                    i += 1
                    flag_jumps_back = True
                # Compare the safety radius with the critical distance for the
                # closest edge (forward)
                while not approved_distance(aux_node_forward, safety_radius,
                                            vertices, 'forward', verbose):
                    # Update the node, jumps and flag
                    aux_node_forward = aux_node_forward.nref
                    i += 1
                    flag_jumps_back = True
                # Get coordinates
                vertex_start = vertices[aux_node_back.pref.item]
                vertex_mid1 = vertices[aux_node_back.item]
                vertex_mid2 = vertices[aux_node_forward.item]
                vertex_end = vertices[aux_node_forward.nref.item]
                # If jump then sharpen the intersection
                if flag_jumps_back or flag_jumps_forward:
                    vertex_mid1 = sharpen(vertex_start, vertex_mid1,
                                          vertex_mid2, vertex_end)
                # Get the new vertices in this sector
                sector = intersect(vertex_start, vertex_mid1, vertex_end,
                                   safety_radius)
                # Assign the new intersection value to new_vertices
                new_vertices.append(sector[1])
                # Move the milestone_node and update the jump
                milestone_node = aux_node_forward.nref
                i += 1
            # Add the first vertex at the end to close the cell 
            new_vertices.append(new_vertices[0])
        # Check if target point is in/out of BVC
        bbPath = path.Path(np.array(new_vertices))
        inside = bbPath.contains_point(target[counter])
        if inside:
            closest_point = target[counter]
            closer_edges.extend([[0,0],[0,0]])
        else:
            # Calculate the closest point using the CVXPY Solver
            if optimize:
                closest_point = solver.optimizer(robots[counter],
                                                 target[counter],
                                                 np.delete(robots, counter,0),
                                                 safety_radius, False, False)
                get_intersection = False
                i = 0
                while not get_intersection and i < len(new_vertices)-1:
                    if isBetween(new_vertices[i+1], new_vertices[i],
                                 closest_point):
                        get_intersection = True
                        closer_edges.extend([new_vertices[i+1],
                                             new_vertices[i]])
                    else:
                        i += 1
                if not get_intersection:
                    closer_edges.extend([[0,0],[0,0]])
            # Calculate the closest point using the Analytical Geometric
            # Algorithm
            else:
                edge_points = perpendicular_distance(target[counter],
                                                     new_vertices)
                closest_point = search_closest_point(np.array(target[counter]),
                                                     np.array(new_vertices
                                                              +edge_points))
                if verbose:
                    print('Closest point candidates:',new_vertices+edge_points)
                    print('Lenght:',len(new_vertices+edge_points))
                    print('THE CLOSEST POINT:',closest_point)
                get_intersection = False
                i = 0
                while not get_intersection and i < len(new_vertices)-1:
                    closest_point = sharpen(target[counter], robots[counter],
                                            new_vertices[i+1],
                                            new_vertices[i])
                    is_in_edge = isBetween(new_vertices[i+1], new_vertices[i],
                                           closest_point)
                    is_in_path = isBetween(target[counter], robots[counter],
                                           closest_point)
                    if is_in_edge and is_in_path:
                        get_intersection = True
                        closer_edges.extend([new_vertices[i+1],
                                             new_vertices[i]])
                    else:
                        i += 1
                if not get_intersection:
                    closer_edges.extend([[0,0],[0,0]])
            if verbose:
                print('Closest point:',closest_point)
        # Store the closest point
        new_robots.append(closest_point)
        # Save the new vertices
        set_of_new_vertices.append(new_vertices)
        if verbose:
            print('The new vertices are:')
            for i in range(len(new_vertices)):
                print(new_vertices[i])
            print('---------------------------------------------------------')
        if plotting:
            # Set the color
            cell_color = palette[counter]
            patches = []
            plt.xlim(-5.6*(len(robots)-1),5.6*(len(robots)-1))
            plt.ylim(-4*(len(robots)-1),4*(len(robots)-1))
            #plt.xlim(-1.05,1.05); plt.ylim(-0.75,0.75)
            # Display edges
            for i in range(len(new_vertices)-1):
                plt.plot([new_vertices[i][0], new_vertices[i+1][0]],
                         [new_vertices[i][1], new_vertices[i+1][1]],
                         color = cell_color, linestyle = '-')
            # Plot robots, target, line to target,robot path and new direction
            plt.plot(robots[counter,0], robots[counter,1],
                     color = cell_color, marker = 'o')
            plt.plot(target[counter,0], target[counter,1], color = cell_color,
                     marker = 'o')
            plt.plot([robots[counter,0], target[counter,0]],
                     [robots[counter,1], target[counter,1]],
                     color = 'gray', linestyle = '--')
            # BVC Cell inside
            polygon = Polygon(np.array(new_vertices), True)
            patches.append(polygon)
            p = PatchCollection(patches, alpha=0.1)
            p.set_color(cell_color)
            ax.add_collection(p)
            # Plot path
            for i in range(len(robots_path)-1):
                inst_start = robots_path[i][counter]
                inst_end = robots_path[i+1][counter]
                plt.plot([inst_start[0], inst_end[0]], [inst_start[1],
                                                        inst_end[1]],
                         color = cell_color, linestyle = '-')
            #plt.plot()
            plt.plot([robots[counter,0], closest_point[0]],
                     [robots[counter,1], closest_point[1]],
                     'k-')
            # Display the Voronoi Diagram
            for arista in edges:
                x_values = [vertices[arista[0],0], vertices[arista[1],0]]
                y_values = [vertices[arista[0],1], vertices[arista[1],1]]
                plt.plot(x_values, y_values, color = 'gray', linestyle = '-')
        # Increase the counter
        counter += 1
    return [np.array(new_robots), np.array(set_of_new_vertices),
            np.array(closer_edges)]